# 6. Exercise 

Session 6 consists only of an exercise, where we combine content from previous sessions into a larger program.  

## Intro
The code we deal with was originally used on a project, but has for the purpose been turned into an exercise.

The code deals with post processing of results from a structural Finite Element calculation. The output is shear forces in beam elements and we will filter the results and plot each Construction Stage with its forces into one figure per Stage.

The model is seen in the image below. It has three shear keys "BS101", "BS201" and "BS301" which extend transversely across the base slab.
Each shear key consists of many small beam elements which span the gap of a movement joint between two slab parts. 

It is the forces in these beam elements that we will work with and plot to see how they vary across the slab in each stage.

![sketch](shear_key_sketch.png)

The end goal of this plotting procedure is to give a quick overview of the results after a calculation has finished, and to be able to flip through the Construction Stages to easily compare them. 

There are in total 56 Construction Stages in the dataset and three different shear keys, resulting in 168 plots.

Each plot will look something like this, which shows the forces in shear key "BS301" for Construction Stage number 4069: 

![plot](BS301_LC4069.png)

Some plots will be almost empty as loads are close to zero in some Stages. 

> **Note:** Understanding the structural context of the dataset is not important for solving the exercise. The same concepts could be used for all other types of datasets. If you will, you can look at it as generic data which is to be filtered and plotted. 

### The dataset
The dataset is called `shear_keys_base_slab_v20.txt`. To get the file go [here](https://raw.githubusercontent.com/Python-Crash-Course/Python101/master/Session%206%20-%20Practical_Exercise/shear_keys_base_slab_v20.txt), right-click and choose "save as". Select `.txt` as file format. Save the file in the same folder as your script for this exercise. 


## The exercise
As stated, this little program was originally used on a project. The general structure of the script is given below and provides the basis for the exercise. 

***Many lines of code from the original script have been removed and the exercise consists of filling them in again.***

All code comments from the original script have been retained as guidance through the exercise. 

The problem is partly about reading and understanding already written code and partly about writing code yourself. 

Reading other people's code plays a big role when collaboration on programming projects, and it's sometimes harder than writing the code yourself. Thus, it's a good exercise to get some exposure to this.

Before starting, open the dataset file `shear_keys_base_slab_v20.txt`. Take a brief look at the contents of the file to get a feel for what you are working with. 

Copy this directly into your editor to use as a guide through the exercise. 

---
~~~python
# Import libraries 
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# Set style for matplotlib plots
plt.style.use('seaborn-whitegrid')    

# Dictionary for mapping beam element numbers to user chosen shear key names
shear_keys = {   
              # Shear key in Base Slab 101
              'BS101': range(10101, 10199),  
    
              # Shear key in Base Slab 201
              'BS201': range(20101, 20199),  
    
              # Shear key in Base Slab 301 
              'BS301': range(30101, 30214),  
}   
    
# Set name of file that contains the dataset
file_name = 'shear_keys_base_slab_v20.txt'

# Read dataset from text file into dataframe, save it as `df`
# <Code here!>

# Extract version number from file name as 'vXX'
# (assume the last 6 characters will always be '...vXX.txt')
# <Code here!>

# Print the head of the dataframe to check it
# <Code here!>

# Construct a dictionary that maps load case numbers to titles (dict auto removes duplicates)
lc_no_to_title_map = dict(zip(df['LC'], df['LC-title']))    
    
# Loop over all shear key names and their corresponding element numbers 
for shear_key, element in shear_keys.items():
    
    # Loop over all load cases, create plots and save them to a png-file
    for lc in df['LC'].unique():

        # Get title of current load case from mapping dictionary
        # <Code here!>    (see hint 1 below)
        
        # Filter dataframe based on load case and element numbers in shear key
        # <Code here!>    (see hint 2 below)
        
        # Create figure
        # <Code here!> 
        
        # Create x-values for plot as numbers running from 1 to length of y-values
        # <Code here!> 
        
        # Create y-values for plot as shear forces vx
        # <Code here!> 
        
        # Extract indices where y-values are negative and positive, respectively
        idx_neg = np.where(y<0)
        idx_pos = np.where(y>=0)
        
        # Extract x-values where y-values are negative and positive, respectively
        x_neg, x_pos = np.take(x, idx_neg)[0], np.take(x, idx_pos)[0]
        
        # Extract y-values where y-values are negative and positive, respectively
        y_neg, y_pos = np.take(y, idx_neg)[0], np.take(y, idx_pos)[0]
        
        # Fill between y=0 and the lines where y-values are negative and positive, respectively 
        # <Code here!> 
  
        # Set titles and x- and y-labels
        # <Code here!>          
                            
        # Save figure to png-file with meaningful name that varies in every loop
        # <Code here!>
~~~
---
### The hints below refer to the comments in the code above.

* **Hint 1:** The dictionary `lc_no_to_title_map` has load case numbers as keys and the corresponding titles as values. Use this to get the load case title from inside the loop.

* **Hint 2:** Be sure to save the filtered DataFrame to a new variable in every loop. If it is saved to a variable of the same name it will be mutated in every loop and quickly end up empty. This way, the DataFrame remains "fresh" when entering each loop. 

### Looping over dictionary items
The outer loop iterates over the key/value pairs of the dictionary called `shear_keys`. The key/value pairs in a dictionary are referred to as its **items**.

`shear_keys.items()` returns the key and value in each loop:

---
```python
for key, value in shear_keys.items():
    print(key, value)
```
would print:

```
BS101 range(10101, 10199)
BS201 range(20101, 20199)
BS301 range(30101, 30214)
```
----

This functionality is equivalent to using `zip()` with lists of keys and values as arguments:

```python
for key, value in zip(shear_keys.keys(), shear_keys.values()):
    print(key, value)
```
which would print exactly the same. 
The built-in dictionary class in Python just has a method for creating this common type of iteration so it is more readable using `dict.items()`.

### Some improvements

* Comparison between the plots when flipping through them could be improved by having the same limits for the y-axis on all plots. This can be set by `ax.set_ylim(bottom_limit, top_limit)`. If any of them are left undefined they will be auto adjusted by default.

Source: https://matplotlib.org/3.1.1/api/_as_gen/matplotlib.axes.Axes.set_ylim.html


* The function below can find the indices of the peak values, which can be used to annotate the key points to make the plot easier to read.  

---
~~~python
def find_local_extrema(y_curve):
    '''
    Return indices of all local extrema for the given sequence of values. Indices are sorted in
    ascending format with no distinction between local maximum and minimum.
    '''
    local_max, _ = find_peaks(y_curve, height=0)
    local_min, _ = find_peaks(-y_curve, height=0)
    return sorted( np.append(local_min, local_max) )    
~~~
---

Prior to running the function, `find_peaks` from the `scipy` library must be imported: `from scipy.signal import find_peaks`

After having found the extrema values, they can be annotated like so:

---
~~~python
for extr_val in extrema_values:
    ax.annotate(f'{y[extr_val]:.0f}', xy=(x[extr_val], y[extr_val]), xytext=(x[extr_val], y[extr_val]))
~~~
---